In [ ]:
# Importing standard Qiskit libraries and configuring account
from qiskit import *
from qiskit import IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import style
style.use('bmh')

from PIL import Image
import math

In [ ]:
crop_size = 2
img_dir = "imgs/elephant.png"

data_qb = int(math.log2(crop_size**2))
anc_qb = 1
total_qb = data_qb + anc_qb
print(f"Total Qubits: {total_qb}")

os.makedirs("result_image_device_Simple", exist_ok=True)

Crop the image

In [ ]:


# Function for plotting the image using matplotlib
def plot_image(img, i, title: str):
    plt.title(title)
    plt.xticks(range(img.shape[0]))
    plt.yticks(range(img.shape[1]))
    
    plt.imshow(img, extent=[0, img.shape[0], img.shape[1], 0], cmap='binary')
    plt.savefig(f"result_image_device_Simple/{title}_{i}.jpg")
    plt.close()
    # plt.show()
    
    np.save(f"result_image_device_Simple/{title}_{i}.npy", img)
    
# plot_image(image, 'Original Image')


In [ ]:
image = np.array(Image.open(img_dir).convert('L'))
# image = image[136:152, 79:95]
# image = np.array([[0, 0, 0, 0],
#                   [0, 1, 1, 0],
#                   [0, 1, 1, 0],
#                   [0, 0, 0, 0]])

image = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 0, 0, 0, 0, 1, 0],
                  [0, 1, 0, 1, 1, 0, 1, 0],
                  [0, 1, 0, 1, 1, 0, 1, 0],
                  [0, 1, 0, 0, 0, 0, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0]])

plt.imshow(image, cmap='binary')
plt.show()
print(f"Original Image Shape: {image.shape}")

original_img = [image.shape[0], image.shape[1]]
times = int(image.shape[0]/crop_size)
crop_image = np.zeros((times**2, crop_size, crop_size))
for i in range(times):
    for j in range(times):
        crop_image[i*times+j] = image[i*crop_size:(i+1)*crop_size, j*crop_size:(j+1)*crop_size]
image = crop_image

print(f"Cropped Image Shape: {image.shape}")


In [ ]:
# Convert the raw pixel values to probability amplitudes
def amplitude_encode(img_data):
    
    # Calculate the RMS value
    rms = np.sqrt((np.sum(np.square(img_data, dtype='float64'), axis=None)))
    
    # Create normalized image
    image_norm = []
    for arr in img_data:
        for ele in arr:
            image_norm.append(ele / rms)
    # Return the normalized image as a numpy array
    return np.array(image_norm)

# Get the amplitude ancoded pixel values
# Horizontal: Original image
image_norm_h = np.asarray([amplitude_encode(image[i]) for i in range(image.shape[0])])

# Vertical: Transpose of Original image
image_norm_v = np.asarray([amplitude_encode(image[i].T) for i in range(image.shape[0])])

In [ ]:
# Initialize the amplitude permutation unitary
D2n_1 = np.roll(np.identity(2**total_qb), 1, axis=1)

In [ ]:
def generateCircuit(image_norm_h, image_norm_v):

    # Create the circuit for horizontal scan
    qc_h = QuantumCircuit(total_qb)
    qc_h.initialize(image_norm_h, range(1, total_qb))
    qc_h.h(0)
    # qc_h.unitary(D2n_1, range(total_qb))
    
    # Decrement gate - START
    qc_h.x(0)
    qc_h.cx(0, 1)
    qc_h.ccx(0, 1, 2)
    # Decrement gate - END
    
    qc_h.h(0)
    qc_h.measure_all()
    # display(qc_h.draw('mpl', fold=-1))

    # Create the circuit for vertical scan
    qc_v = QuantumCircuit(total_qb)
    qc_v.initialize(image_norm_v, range(1, total_qb))
    qc_v.h(0)
    # qc_v.unitary(D2n_1, range(total_qb))
    
    # Decrement gate - START
    qc_v.x(0)
    qc_v.cx(0, 1)
    qc_v.ccx(0, 1, 2)
    # Decrement gate - END
    
    qc_v.h(0)
    qc_v.measure_all()
    # display(qc_v.draw('mpl', fold=-1))

    # Combine both circuits into a single list
    circ_list = [qc_h, qc_v]
    
    return qc_h, qc_v, circ_list

In [ ]:
def QuantumSimulate(qc_h, qc_v, circ_list):
    # Simulating the cirucits
    back = Aer.get_backend('statevector_simulator')
    results = execute(circ_list, backend=back).result()
    sv_h = results.get_statevector(qc_h)
    sv_v = results.get_statevector(qc_v)

    from qiskit.visualization import array_to_latex
    print('Horizontal scan statevector:')
    # display(array_to_latex(sv_h[:30], max_size=30))
    print()
    print('Vertical scan statevector:')
    # display(array_to_latex(sv_v[:30], max_size=30))
    
    return sv_h, sv_v

In [ ]:
def scanEdge(sv_h, sv_v):
    edge_scan_h = np.array([sv_h[f'{2*i+1:0{total_qb}b}'].real for i in range(2**data_qb)]).reshape(crop_size, crop_size)
    edge_scan_v = np.array([sv_v[f'{2*i+1:0{total_qb}b}'].real for i in range(2**data_qb)]).reshape(crop_size, crop_size).T

    return edge_scan_h, edge_scan_v

In [ ]:
Total_scan_h = []
Total_scan_v = []
circ_list_all = []
for i in range(image_norm_h.shape[0]):
    # print(f"Idx: {i}")
    qc_h, qc_v, circ_list = generateCircuit(image_norm_h[i], image_norm_v[i])
    circ_list_all.append(circ_list)
    

In [ ]:
display(circ_list_all[0][0].draw('mpl', fold=-1))
display(circ_list_all[0][1].draw('mpl', fold=-1))

In [ ]:
# Load the IBMQ account
from qiskit.tools.monitor import job_monitor
from qiskit import IBMQ
IBMQ.load_account()

# Get the provider and backend
provider = IBMQ.get_provider(hub='ibm-q', group='open')
backend = provider.get_backend('ibm_kyoto')

results = []
edge_scan_result = []

Total_scan_h = []
Total_scan_v = []

for i, circ in enumerate(circ_list_all):
    # Transpile the circuits for optimized execution on the backend
    qc_h_t = transpile(circ[0], backend=backend, optimization_level=3)
    qc_v_t = transpile(circ[1], backend=backend, optimization_level=3)
    circ_list_t = [qc_h_t, qc_v_t]
    job = execute(circ_list_t, backend=backend, shots=4096)
    job_monitor(job)
    result = job.result()
    counts_h = result.get_counts(qc_h_t)
    counts_v = result.get_counts(qc_v_t)
    results.append([counts_h, counts_v])
    edge_scan_small_h = np.array([counts_h[f'{2*i+1:0{total_qb}b}'] for i in range(2**data_qb)]).reshape(crop_size, crop_size)
    edge_scan_small_v = np.array([counts_v[f'{2*i+1:0{total_qb}b}'] for i in range(2**data_qb)]).reshape(crop_size, crop_size).T
    Total_scan_h.append(edge_scan_small_h) #[16, 128, 128]
    Total_scan_v.append(edge_scan_small_v)
    edge_scan_result.append([edge_scan_small_h, edge_scan_small_v])

    # npy saved in plot_image
    plot_image(edge_scan_small_h, i=i, title='Horizontal')
    plot_image(edge_scan_small_v, i=i, title='Vertical')
    
    Add = edge_scan_small_h + edge_scan_small_v
    plot_image(Add, i=i, title='Add')
    

In [ ]:
print(counts_h)

In [ ]:
print(edge_scan_small_h)
print(edge_scan_small_v)

In [ ]:
Total_scan_h = np.asarray(Total_scan_h)   
Total_scan_v = np.asarray(Total_scan_v)
Result_h = np.zeros((original_img[0], original_img[1]))
Result_v = np.zeros((original_img[0], original_img[1]))

for i in range(times):
    for j in range(times):
        Result_h[crop_size*i:crop_size*(i+1), crop_size*j:crop_size*(j+1)] = Total_scan_h[i*(times)+j,:,:] 
        Result_v[crop_size*i:crop_size*(i+1), crop_size*j:crop_size*(j+1)] = Total_scan_v[i*(times)+j,:,:]

plot_image(Result_h, i=i, title='Result_Horizontal')
plot_image(Result_v, i=i, title='Result_Vertical')

In [ ]:
All = Result_h + Result_v
plot_image(All, i=i, title='All')

In [ ]:
# Classical postprocessing for plotting the output

# Defining a lambda function for
# thresholding to binary values
# thr = 2e-20
# threshold = lambda amp: (amp > thr or amp < -thr)

# Selecting odd states from the raw statevector and
# reshaping column vector of size 64 to an 8x8 matrix
# edge_scan_h = np.abs(np.array([1 if threshold(sv_h[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(image.shape[0], image.shape[1])
# edge_scan_v = np.abs(np.array([1 if threshold(sv_v[2*i+1].real) else 0 for i in range(2**data_qb)])).reshape(image.shape[0], image.shape[1]).T


# Plotting the Horizontal and vertical scans
# plot_image(edge_scan_h, 'Horizontal scan output')
# plot_image(edge_scan_v, 'Vertical scan output')

# plot_image(edge_scan_h, 'Horizontal scan output')
# plot_image(edge_scan_v, 'Vertical scan output')

In [ ]:
# Combining the horizontal and vertical component of the result
# edge_scan_sim = edge_scan_h  | edge_scan_v
# edge_scan_sim = edge_scan_h*0.5 + edge_scan_v*0.5

# # Plotting the original and edge-detected images
# plot_image(image, 'Original image')
# plot_image(edge_scan_sim, 'Edge Detected image')